## Summing all the prime numbers below a given number

In [18]:
#3.2d
import os
import sys
import numba

number = 2_500_000  # valor por defecto

for arg in sys.argv[1:]:
    try:
        number = int(arg)
        break
    except ValueError:
        pass


ncores = int(os.environ.get("SLURM_CPUS_PER_TASK", 1))

print("Number =", number)
print("Cores =", ncores)


Number = 2500000
Cores = 1


In [19]:
import time

# Simple code

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

number = 2_500_000
suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 4.787564913431804
The prime sum below  2500000 is  219697708195  and the time taken is 4.71 s ± 15 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [20]:
#3.2a
from numba import njit

@njit
def if_prime_numba(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes_numba(x):
    result = 0
    for i in range(x):
        result += if_prime_numba(i)
    return result

In [21]:
number = 2_500_000
N = 3

start = time.time()
for i in range(N):
    suma = sum_primes_numba(number)
stop = time.time()

tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 0.2237086296081543


In [22]:
#3.2b

import multiprocessing as mp

@njit
def if_prime_numba(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x


@njit
def sum_primes_range(start, end):
    result = 0
    for i in range(start, end):
        result += if_prime_numba(i)
    return result


In [23]:
number = 2_500_000
nproc = 4
chunk = number // nproc

ranges = [(i * chunk, (i + 1) * chunk) for i in range(nproc)]

In [24]:
N = 3

start = time.time()
for _ in range(N):
    with mp.Pool(processes=nproc) as pool:
        partial_sums = pool.starmap(sum_primes_range, ranges)
    suma = sum(partial_sums)
stop = time.time()

tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 0.2162634531656901


In [25]:
#3.2c
from numba import prange
numba.set_num_threads(4)


@njit
def if_prime_numba(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x


@njit(parallel=True)
def sum_primes_prange(x):
    result = 0
    for i in prange(x):
        result += if_prime_numba(i)
    return result

In [26]:
number = 2_500_000
N = 3

start = time.time()
for _ in range(N):
    suma = sum_primes_prange(number)
stop = time.time()

tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)
print("Num threads:", numba.get_num_threads())

The prime sum below  2500000 is  219697708195  and the time taken is 0.18657159805297852
Num threads: 4


Number = 1000000
Cores = 1
The prime sum below  2500000 is  219697708195  and the time taken is 8.485645453135172
The prime sum below  2500000 is  219697708195  and the time taken is 8.46 s ± 61.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  2500000 is  219697708195  and the time taken is 1.9007132053375244
The prime sum below  2500000 is  219697708195  and the time taken is 0.363190491994222
The prime sum below  2500000 is  219697708195  and the time taken is 0.3876919746398926
Num threads: 4
Number = 10000000
Cores = 1
The prime sum below  2500000 is  219697708195  and the time taken is 8.29316751162211
The prime sum below  2500000 is  219697708195  and the time taken is 8.09 s ± 1.01 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  2500000 is  219697708195  and the time taken is 0.7754298051198324
The prime sum below  2500000 is  219697708195  and the time taken is 0.3585902849833171
The prime sum below  2500000 is  219697708195  and the time taken is 0.3713420232137044
Num threads: 4


Number = 1000000
Cores = 2
The prime sum below  2500000 is  219697708195  and the time taken is 8.276272535324097
The prime sum below  2500000 is  219697708195  and the time taken is 8.05 s ± 3.01 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  2500000 is  219697708195  and the time taken is 1.8811206022898357
The prime sum below  2500000 is  219697708195  and the time taken is 0.36237557729085285
The prime sum below  2500000 is  219697708195  and the time taken is 0.38861457506815594
Num threads: 4
Number = 10000000
Cores = 2
The prime sum below  2500000 is  219697708195  and the time taken is 8.269906044006348
The prime sum below  2500000 is  219697708195  and the time taken is 8.05 s ± 2.35 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  2500000 is  219697708195  and the time taken is 0.5826252301534017
The prime sum below  2500000 is  219697708195  and the time taken is 0.34553805987040204
The prime sum below  2500000 is  219697708195  and the time taken is 0.3704032103220622
Num threads: 4


Number = 1000000
Cores = 4
The prime sum below  2500000 is  219697708195  and the time taken is 8.29414145151774
The prime sum below  2500000 is  219697708195  and the time taken is 8.02 s ± 3.77 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  2500000 is  219697708195  and the time taken is 0.6844205061594645
The prime sum below  2500000 is  219697708195  and the time taken is 0.3463159402211507
The prime sum below  2500000 is  219697708195  and the time taken is 0.3741167386372884
Num threads: 4
Number = 10000000
Cores = 4
The prime sum below  2500000 is  219697708195  and the time taken is 8.287953535715738
The prime sum below  2500000 is  219697708195  and the time taken is 8.24 s ± 198 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  2500000 is  219697708195  and the time taken is 0.5760743618011475
The prime sum below  2500000 is  219697708195  and the time taken is 0.34850525856018066
The prime sum below  2500000 is  219697708195  and the time taken is 0.38180263837178546
Num threads: 4


Number = 1000000
Cores = 8
The prime sum below  2500000 is  219697708195  and the time taken is 8.362866957982382
The prime sum below  2500000 is  219697708195  and the time taken is 8.31 s ± 85.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  2500000 is  219697708195  and the time taken is 0.6936843395233154
The prime sum below  2500000 is  219697708195  and the time taken is 0.3501489957173665
The prime sum below  2500000 is  219697708195  and the time taken is 0.3772389888763428
Num threads: 4
Number = 10000000
Cores = 8
The prime sum below  2500000 is  219697708195  and the time taken is 8.664463122685751
The prime sum below  2500000 is  219697708195  and the time taken is 8.13 s ± 51.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
The prime sum below  2500000 is  219697708195  and the time taken is 0.6795173486073812
The prime sum below  2500000 is  219697708195  and the time taken is 0.3488360246022542
The prime sum below  2500000 is  219697708195  and the time taken is 0.3694790204366048
Num threads: 4

La suma de números primos se ha ejecutado en el nodo mendel utilizando cuatro enfoques distintos, comparando sus tiempos de ejecución para number = 10⁶ y number = 10⁷, así como el efecto del número de núcleos asignados.

En primer lugar, el código original en Python presenta los peores tiempos de ejecución. Para number = 10⁶, el tiempo ronda los 8.3–8.5 segundos, mientras que para number = 10⁷ el tiempo se mantiene también en torno a los 8–9 segundos, mostrando claramente la ineficiencia del código secuencial interpretado para este tipo de cálculos intensivos.

La versión secuencial con Numba (@njit) mejora notablemente el rendimiento. Para number = 10⁶, el tiempo se reduce aproximadamente a 1.9 segundos, y para number = 10⁷ a alrededor de 0.75 segundos, lo que demuestra el impacto positivo de la compilación JIT incluso sin paralelización.

En el caso de la paralelización con multiprocessing.Pool, se observa una mejora adicional respecto a la versión secuencial, con tiempos alrededor de 0.36–0.39 segundos para number = 10⁶. Sin embargo, el speedup no escala de forma ideal al aumentar el número de núcleos, debido al overhead asociado a la creación de procesos y a la comunicación entre ellos.

Por último, la versión paralela con Numba usando prange es la que ofrece el mejor comportamiento global. Para number = 10⁶, los tiempos se sitúan aproximadamente entre 0.35 y 0.38 segundos, y para number = 10⁷ entre 0.57 y 0.68 segundos, dependiendo del número de núcleos asignados. Al aumentar los cores de 1 a 4 se observa una mejora en el tiempo de ejecución, aunque a partir de 4–8 núcleos el speedup se estabiliza, lo que indica que el coste del paralelismo empieza a compensar la ganancia obtenida.

En conjunto, los resultados muestran que Numba con prange es la opción más eficiente para este problema en el entorno HPC utilizado, ya que combina compilación JIT y paralelización con un overhead reducido, ofreciendo los mejores tiempos de ejecución.